In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [3]:
data_1 = pd.read_csv('lin_reg_1.txt')

xx = data_1['population']
yy = data_1['profit']

data_1.insert(0, 'ones', 1)
data_1.head()

,ones,population,profit
0,1,6.1101,17.5920
1,1,5.5277,9.1302
2,1,8.5186,13.6620
3,1,7.0032,11.8540
4,1,5.8598,6.8233


In [4]:
cols = data_1.shape[1] # column number

X = np.matrix(data_1.iloc[:, 0:cols-1].values)   # 1's + population
Y = np.matrix(data_1.iloc[:,cols-1:cols].values) # profit
theta = np.matrix(np.zeros(2))

# parameters for gradient descent
m = len(Y)
alpha = 0.01
iters = 1200

print('dimentions of X: {}, dimentions of Y: {}'.format(X.shape,Y.shape))

dimentions of X: (97, 2), dimentions of Y: (97, 1)


In [7]:
def computeCost(X, Y, theta, m):
    difference = (X * theta.T) - Y
    J = ( 1/(2*m) * np.power(difference, 2) ).sum()
    return J

J = computeCost(X, Y, theta, m)
J_test = computeCost(X, Y, np.matrix([-1, 2]), m)
print("Cost function for theta=[0;0]  is {}".format(J))
print("Cost function for theta=[-1;2] is {}".format(J_test))

Cost function for theta=[0;0]  is 32.07273387745567
Cost function for theta=[-1;2] is 54.24245508201237


In [29]:
def gradientDescent(X, Y, theta, alpha, iters):
    temp = np.matrix(np.zeros(theta.shape))
    parameters = theta.shape[1]
    cost = np.zeros(iters)

    for i in range(iters):
        difference = (X * theta.T) - Y
        for j in range(parameters):
            term = np.multiply(difference, X[:,j])
            temp[0,j] = theta[0,j] - ( alpha/len(X) * np.sum(term) )
            theta = temp
            cost[i] = computeCost(X, Y, theta, m)

    return theta, cost

theta_update, cost = gradientDescent(X, Y, theta, alpha, iters)
print("After {} iterations, theta = {}".format(iters, theta_update))

After 1200 iterations, theta = [[-3.43962171  1.14720751]]
